In [3]:
import pandas as pd

## Chargement des données

In [12]:
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Query_poly_20201211153346.xlsx", converters={'Matricule':str}) # effectif présent
infoc = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Individus_liste_info_complementaires.xlsx", converters={'Code de la ressource':str}) # données extenes

## Contrôle des données

### Contrôle login SAP

In [14]:
fafr = fafr.merge(infoc[["Code de la ressource", "Login SAP"]], left_on=["Matricule"], right_on=["Code de la ressource"], how='left')

In [24]:
fafr["Login SAP vide"] = ""
fafr.loc[fafr["Login SAP"].isna()==True, "Login SAP vide"] = "Oui"
fafr_sap_ko = fafr.loc[fafr["Login SAP vide"]=="Oui"]

## Export

### Export login SAP KO

In [26]:
fafr_sap_ko[["Matricule","Nom", "Prénom", "Etablissement", "Login SAP"]].to_excel(r'C:\Users\sgasmi\Desktop\Export Python\SAP_KO.xlsx', index=False)